In [8]:
import argparse
import numpy as np
import soundfile as sf


def generate_noise_flac(
    output_path: str,
    seconds: float,
    sample_rate: int = 44100,
    channels: int = 2,
    amplitude: float = 0.2,
    seed: int | None = None,
) -> None:
    if seconds <= 0:
        raise ValueError("seconds must be > 0")
    if sample_rate <= 0:
        raise ValueError("sample_rate must be > 0")
    if channels <= 0:
        raise ValueError("channels must be > 0")
    if not (0.0 < amplitude <= 1.0):
        raise ValueError("amplitude must be in (0, 1]")

    frames = int(round(seconds * sample_rate))
    rng = np.random.default_rng(seed)

    # Float32 noise in [-1, 1], scaled down by amplitude to avoid clipping.
    audio = (rng.uniform(-1.0, 1.0, size=(frames, channels)).astype(np.float32) * amplitude)

    # Write as 16-bit PCM inside a FLAC container.
    sf.write(output_path, audio, sample_rate, format="FLAC", subtype="PCM_16")


def main() -> None:
    p = argparse.ArgumentParser(description="Generate random-noise FLAC of N seconds.")
    p.add_argument("seconds", type=float, help="Length in seconds (e.g., 10 or 2.5)")
    p.add_argument("-o", "--output", default="noise.flac", help="Output path (default: noise.flac)")
    p.add_argument("-r", "--sample-rate", type=int, default=44100, help="Sample rate (default: 44100)")
    p.add_argument("-c", "--channels", type=int, default=2, help="Channels (default: 2)")
    p.add_argument("-a", "--amplitude", type=float, default=0.2, help="Amplitude in (0,1] (default: 0.2)")
    p.add_argument("--seed", type=int, default=None, help="RNG seed (optional)")
    args = p.parse_args()

    
    print(f"Wrote {args.output} ({args.seconds:.3f}s, {args.sample_rate} Hz, {args.channels}ch)")

In [9]:
from tqdm.notebook import tqdm

In [10]:
twenty_minutes = 20 * 60
ten_minutes = 5 * 60
for seconds in tqdm(range(ten_minutes, twenty_minutes + 30, 30)):
    generate_noise_flac(
        output_path=f"./fake_{seconds}.flac",
        seconds=seconds,
        sample_rate=16000,
        channels=2,
        amplitude=0.3,
        seed=None,
    )


  0%|          | 0/31 [00:00<?, ?it/s]

In [7]:
!rm fake_*.flac